In [4]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from transformers import AutoModelForImageSegmentation

In [6]:
pip install kornia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.5 MB/s eta 0:00:00


In [7]:
model = AutoModelForImageSegmentation.from_pretrained('briaai/RMBG-2.0', trust_remote_code=True)
torch.set_float32_matmul_precision(['high', 'highest'][0])
model.to('cuda')
model.eval()

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


model.safetensors:   0%|          | 0.00/885M [00:00<?, ?B/s]

BiRefNet(
  (bb): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=192, out_features=576, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=192, out_features=192, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=192, out_features=768, bias=T

In [9]:
!unzip /content/test.zip

Archive:  /content/test.zip
  inflating: test/1.jpg              
  inflating: test/10.webp            
  inflating: test/11.webp            
  inflating: test/12.jpg             
  inflating: test/13.webp            
 extracting: test/14.webp            
  inflating: test/15.jpg             
  inflating: test/16.jpg             
  inflating: test/16.webp            
  inflating: test/17.jpg             
  inflating: test/18.jpg             
  inflating: test/19.jpg             
 extracting: test/2.webp             
  inflating: test/20.jpg             
  inflating: test/21.jpg             
  inflating: test/22.jpg             
  inflating: test/23.jpg             
  inflating: test/24.jfif            
 extracting: test/25.webp            
 extracting: test/26.webp            
  inflating: test/27.webp            
  inflating: test/28.jpg             
  inflating: test/29.jpg             
  inflating: test/3.jpg              
  inflating: test/30.jpg             
  inflating: test/31.j

In [14]:
# Data settings
image_size = (1024, 1024)
transform_image = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [25]:
import os

for _, _, files in os.walk('/content/test/sint2'):
  i = 1
  for file in files:

    image = Image.open(f'/content/test/sint2/{file}')
    input_images = transform_image(image).unsqueeze(0).to('cuda')

  # Prediction
    with torch.no_grad():
      preds = model(input_images)[-1].sigmoid().cpu()
      pred = preds[0].squeeze()
      pred_pil = transforms.ToPILImage()(pred)
      mask = pred_pil.resize(image.size)
      image.putalpha(mask)

      image.save(f"/content/sim2_without_background/{i}.png")
      i += 1


In [27]:
import zipfile
import os

# Путь к файлу или папке, которые нужно архивировать
file_or_folder = '/content/sim2_without_background'

# Путь к создаваемому архиву
zip_filename = 'sim2_w_b.zip'

# Создание zip архива
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    if os.path.isdir(file_or_folder):
        # Если это папка, добавляем все её файлы
        for foldername, subfolders, filenames in os.walk(file_or_folder):
            for filename in filenames:
                zipf.write(os.path.join(foldername, filename),
                           os.path.relpath(os.path.join(foldername, filename), file_or_folder))
    else:
        # Если это файл, просто добавляем его
        zipf.write(file_or_folder, os.path.basename(file_or_folder))

print(f'Архив {zip_filename} успешно создан!')


Архив sim2_w_b.zip успешно создан!
